In [8]:
import json
from pymongo import MongoClient

#1:Now we try to upload the json to mongodb
cl=MongoClient('mongodb://localhost:27017/')
dab=cl['ev_project']
coll=dab['ev_registrations']
coll.drop()

with open('ev_registrations.json','r') as file:
    json_da=json.load(file)
da=json_da['data']
colm=[col['name']for col in json_da['meta']['view']['columns']]
data_dt=[dict(zip(colm,recd))for recd in da]

#it is also necessary for us to insert it in chunks for better efficiency
cz=1000
total_rec=len(data_dt)
for i in range(0,total_rec,cz):
    ck=data_dt[i:i+cz]
    coll.insert_many(ck)
        


In [13]:
import pandas as pd
from pymongo import MongoClient
from datetime import datetime

cl=MongoClient('mongodb://localhost:27017/')
dab=cl['ev_project']
coll=dab['ev_registrations']

sd=datetime(2023,1,1)
ed=datetime(2025,6,30)
evda=list(coll.find({'Fuel Type':'Electric','Transaction Date':{'$gte':sd.strftime('%Y-%m-%d'),'$lte':ed.strftime('%Y-%m-%d')}}))

df=pd.DataFrame(evda).drop(columns=['_id','sid','id','position','created_at','updated_at','updated_meta','meta','Fiscal Year'],errors='ignore')

df['Counts']=pd.to_numeric(df['Counts'],errors='coerce')
df=df.drop_duplicates()
print(len(df))

print('uniq residential county',df['Residential County'].nunique())
print('unique fuel type',df['Fuel Type'].nunique())
print(df['Counts'].max())  #we try to see the max value of counts

#we try to print only the monthly data instead of daily data
df['Transaction Date']=pd.to_datetime(df['Transaction Date'])
df['YearMonth']=df['Transaction Date'].dt.to_period('M')
df_group=df.groupby(['Residential County','YearMonth','Fuel Type','Primary Use Class'],as_index=False)['Counts'].sum()
df_group['Residential County']=df_group['Residential County'].str.strip()

print('first 5 rows',df_group.head())
df_group.to_csv('ev_cleaned_monthly.csv',index=False)


8797
uniq residential county 39
unique fuel type 1
8448
first 5 rows   Residential County YearMonth Fuel Type  Primary Use Class  Counts
0              Adams   2023-01  Electric         Commercial       1
1              Adams   2023-01  Electric  Passenger Vehicle       1
2              Adams   2023-02  Electric              Truck       1
3              Adams   2023-03  Electric  Passenger Vehicle       1
4              Adams   2023-04  Electric  Passenger Vehicle       4
